In [ ]:
!pip install langchain
!pip install underthesea

#Define function to process data

In [59]:
from underthesea import word_tokenize

from gensim.models import LdaModel
from gensim.corpora import Dictionary
import numpy as np

def save_chunked_paragraph_to_file(file_path, content):
  with open(file_path, 'w') as file:
    file.write(content)

def preprocess_paragraph(paragraph):
  """
  Tiền xử lý văn bản: chuyển đổi văn bản thành một danh sách các từ.
  Args:
  - paragraph: đoạn văn cần tiền xử lý.
  Returns:
  - tokenized_text: danh sách các từ đã được tiền xử lý.
  """
  # tokenized_text = paragraph.lower().split()
  # return tokenized_text
  tokenized_text = word_tokenize(paragraph)
  return tokenized_text

def get_topic_distribution(text, lda_model, dictionary):
  """
  Trả về phân phối chủ đề của một đoạn văn bản.
  Args:
  - text: đoạn văn bản cần xác định phân phối chủ đề.
  - lda_model: mô hình LDA đã được huấn luyện.
  - dictionary: từ điển Gensim.
  Returns:
  - topic_distribution: phân phối chủ đề của đoạn văn bản.
  """
  tokenized_text = preprocess_paragraph(text)
  bow_vector = dictionary.doc2bow(tokenized_text)
  topic_distribution = lda_model.get_document_topics(bow_vector)
  return np.array([topic_prob[1] for topic_prob in topic_distribution])

def kl_divergence(p, q):
  #Nếu P và Q càng giống nhau, KL divergence càng nhỏ.
  return np.exp(-np.sum(p * np.log(p / q)))

def compare_topic_distributions(topic_dist1, topic_dist2, threshold):
  """
  So sánh phân phối chủ đề giữa hai đoạn văn bản.
  Args:
  - topic_dist1: phân phối chủ đề của đoạn văn bản thứ nhất.
  - topic_dist2: phân phối chủ đề của đoạn văn bản thứ hai.
  - threshold: ngưỡng để xác định sự khác biệt lớn.
  Returns:
  - is_different: True nếu có sự khác biệt lớn, False nếu không.
  """
  kl_score = kl_divergence(topic_dist1, topic_dist2)
  # print(kl_score)
  if kl_score > threshold:
      return True
  else:
      return False

#Define function to chunk

In [60]:
from underthesea import sent_tokenize

#is_preview dùng để in ra console trong trường hợp không muốn xuất file output
def chunk_document(file_path, target_dir, window_size = 2, threshold = 0.995, is_preview = False):
  #Đọc file
  document = ''

  with open(file_path, 'r') as file:
    document = file.read()

  #Chia văn bản thành các câu:
  sentences = sent_tokenize(document)

  index = 0
  paragraph_size = 0
  chunk_index = 0

  while index < len(sentences):
    paragraph_size += window_size

    # Lấy các câu trong slide window
    paragraph1 = " ".join(sentences[index:index + paragraph_size])

    if index + paragraph_size > len(sentences):
      if is_preview:
        print("Length paragraph: " + str(paragraph_size) + " sentences")
        print(paragraph1)
        print("----------------------------------------------")
      else:
        chunk_file = target_dir + '/chunk' + str(chunk_index) + '.txt'
        save_chunked_paragraph_to_file(chunk_file, paragraph1)
      chunk_index += 1
      break

    paragraph2 = ""
    if paragraph_size >= window_size * 2:
      paragraph2 = " ".join(sentences[index + paragraph_size: index + paragraph_size + window_size * 2])
    else:
      paragraph2 = " ".join(sentences[index + paragraph_size: index + paragraph_size + window_size])


    # Tiền xử lý văn bản
    preprocessed_documents = [preprocess_paragraph(paragraph1), preprocess_paragraph(paragraph2)]

    # Tạo từ điển
    dictionary = Dictionary(preprocessed_documents)

    # Tạo BoW Corpus
    bow_corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]

    # Huấn luyện mô hình LDA
    lda_model = LdaModel(bow_corpus, num_topics=2, id2word=dictionary, passes=20)

    # So sánh phân phối chủ đề giữa các đoạn văn bản
    topic_distribution1 = get_topic_distribution(paragraph1, lda_model, dictionary)
    topic_distribution2 = get_topic_distribution(paragraph2, lda_model, dictionary)

    # Kiểm tra sự khác biệt giữa hai đoạn văn bản
    if compare_topic_distributions(topic_distribution1, topic_distribution2, threshold):
      if is_preview:
        print("Length paragraph: " + str(paragraph_size) + " sentences")
        print(paragraph1)
        print("----------------------------------------------")
      else:
        #Lưu file
        chunk_file = target_dir + '/chunk' + str(chunk_index) + '.txt'
        save_chunked_paragraph_to_file(chunk_file, paragraph1)
      chunk_index += 1
      # Tiến tới so sánh tiếp theo
      index += paragraph_size
      paragraph_size = 0


# Sample

In [61]:
import os
from pathlib import Path

def chunk_all_file_in_dir(src_dir, target_dir):
  files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]
  for file in files:
    file_path = src_dir + '/' + file
    file_name = os.path.splitext(file)[0]
    output_path = target_dir + '/' + file_name
    # Create the directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)
    chunk_document(file_path, output_path)

chunk_all_file_in_dir('input', 'output')



In [ ]:
chunk_document('input/test2.txt', 'output/test2', is_preview=True)